In [6]:
from src.schemas.thesaurus.mads import SchemaMads

from src.schemas.settings import Settings
from rdflib import URIRef, Namespace, Graph, Literal, BNode
from rdflib.namespace import RDF, RDFS
settings = Settings()

In [2]:
from src.schemas.thesaurus.mads import SchemaMads 
from datetime import datetime

In [1]:
req = {
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        }
    },
    "resource": [
        {
            "type": {
                "value": "http://www.loc.gov/mads/rdf/v1#Authority",
                "label": "Authority"
            }
        },
        {
            "type": {
                "value": "http://www.loc.gov/mads/rdf/v1#Topic",
                "label": "Topic"
            }
        }
    ],
    "isMemberOfMADSCollection": [
        {
            "collection": {
                "value": "subjects",
                "label": "Assuntos"
            }
        }
    ],
    "elementList": [
        {
            "elementType": {
                "value": "http://www.loc.gov/mads/rdf/v1#TopicElement",
                "label": "TopicElement"
            },
            "elementValue": "Entomologia",
            "elementLang": {
                "value": "por",
                "label": "Inglês"
            }
        }
    ],
    "authoritativeLabel": {
        "value": "Entomologia"
    },
    "hasReciprocalAuthority": [
        {
            "authority": {
                "base": "bk",
                "value": "https://bibliokeia.com/authorities/118",
                "label": "Insetos"
            }
        }
    ],
    "hasBroaderAuthority": [
        {
            "authority": {
                "base": "loc",
                "value": "http://id.loc.gov/authorities/subjects/sh85149983",
                "label": "Zoology"
            }
        }
    ],
    "hasNarrowerAuthority": [
        {
            "authority": {
                "base": "loc",
                "value": "http://id.loc.gov/authorities/subjects/sh2012002108",
                "label": "Lepidopterology"
            }
        },
        {
            "authority": {
                "base": "loc",
                "value": "http://id.loc.gov/authorities/subjects/sh85110630",
                "label": "Radioactive tracers in entomology"
            }
        },
        {
            "authority": {
                "base": "loc",
                "value": "http://id.loc.gov/authorities/subjects/sh2012002226",
                "label": "Odonatology"
            }
        },
        {
            "authority": {
                "base": "loc",
                "value": "http://id.loc.gov/authorities/subjects/sh95005625",
                "label": "Paleoentomology"
            }
        },
        {
            "authority": {
                "base": "loc",
                "value": "http://id.loc.gov/authorities/subjects/sh86003985",
                "label": "Forensic entomology"
            }
        },
        {
            "authority": {
                "base": "loc",
                "value": "http://id.loc.gov/authorities/subjects/sh93000054",
                "label": "Urban entomology"
            }
        },
        {
            "authority": {
                "base": "loc",
                "value": "http://id.loc.gov/authorities/subjects/sh2012001930",
                "label": "Dipterology"
            }
        },
        {
            "authority": {
                "base": "loc",
                "value": "http://id.loc.gov/authorities/subjects/sh96005018",
                "label": "Ethnoentomology"
            }
        },
        {
            "authority": {
                "base": "loc",
                "value": "http://id.loc.gov/authorities/subjects/sh2012001929",
                "label": "Myrmecology"
            }
        },
        {
            "authority": {
                "base": "loc",
                "value": "http://id.loc.gov/authorities/subjects/sh2012002193",
                "label": "Coleopterology"
            }
        }
    ],
    "hasCloseExternalAuthority": [
        {
            "base": "id.worldcat.org",
            "label": "Entomology",
            "uri": "http://id.worldcat.org/fast/912735"
        },
        {
            "base": "www.yso.fi",
            "label": "hyönteistiede",
            "uri": "http://www.yso.fi/onto/yso/p16960"
        },
        {
            "base": "datos.bne.es",
            "label": "Entomología",
            "uri": "http://datos.bne.es/resource/XX527256"
        },
        {
            "base": "d-nb.info",
            "label": "Angewandte Entomologie",
            "uri": "https://d-nb.info/gnd/4492833-6"
        },
        {
            "base": "data.bnf.fr",
            "label": "Entomologie",
            "uri": "http://data.bnf.fr/ark:/12148/cb11958162n"
        },
        {
            "base": "purl.org",
            "label": "Entomologia",
            "uri": "http://purl.org/bncf/tid/16915"
        },
        {
            "base": "www.wikidata.org",
            "label": "entomology",
            "uri": "http://www.wikidata.org/entity/Q39286"
        },
        {
            "base": "d-nb.info",
            "label": "Entomologie",
            "uri": "https://d-nb.info/gnd/4152382-9"
        }
    ],
    "identifiersLccn": "sh85044125"
}

In [3]:
request = SchemaMads(**req)
now = datetime.now()
request.adminMetadata.creationDate = now
request.adminMetadata.identifiedBy = 'TESTE'

In [10]:
from src.function.thesaurus.solr.updateSolr import UpdateSolr
from pysolr import Solr



UpdateSolr(request)


NameError: name 'Settings' is not defined

In [11]:
from pysolr import Solr
from src.schemas.settings import Settings
settings = Settings()
solr = Solr(f'{settings.solr}/solr/authority/', timeout=10)

for i in request.hasReciprocalAuthority:
    if i.authority.base == 'bk':
        id = i.authority.value.split("/")[-1]
        res = solr.search(q=f"id:authority#{id}", fl="*,[child]")
        [doc] = res.docs
        print(doc)

{'id': 'authority#118', 'type': ['Authority', 'Topic'], 'creationDate': '2024-06-23T00:00:00Z', 'authority': ['Insetos'], 'isMemberOfMADSCollection': ['subjects'], 'lang': 'Inglês', 'identifiersLccn': ['sh85066670'], 'variant': ['Hexapoda', 'Insecta', 'Pterygota'], 'hasBroaderAuthorityLabels': ['Arthropoda'], 'hasNarrowerAuthorityLabels': ['Psocoptera', 'Embioptera', 'Lice', 'Hemiptera', 'Bloodsucking insects', 'Protura', 'Carrion insects', 'Odonata', 'Apterygota', 'Saproxylic insects', 'Gnats', 'Forest insects', 'Insect pests', 'Strepsiptera', 'Stoneflies', 'Lepidoptera', 'Mantodea', 'Queens (Insects)', 'Electric organs in insects', 'Mayflies', 'Termites', 'Phasmatoptera', 'Predatory insects', 'Beneficial insects', 'Flies', 'Neuroptera', 'Fleas', 'Phytophagous insects', 'Photography of insects', 'Rare insects', 'Classification--Books--Insects', 'Archaeognatha', 'Anoplura', 'Fertilization of plants by insects', 'Thrips', 'Insect societies', 'Beetles', 'Introduced insects', 'Aquatic ins

In [13]:
from pysolr import Solr
solr = Solr(f'{settings.solr}/solr/authority/', timeout=10)

In [14]:
responseSolr = solr.add([doc], commit=True)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":146\n  }\n}'

In [4]:
from src.function.thesaurus.solr.updateSolr import UpdateSolr


UpdateSolr(request)

In [1]:
from rdflib import Graph
graph = Graph()
graph.parse('http://id.loc.gov/authorities/names/no2011070109.rdf')
graph.serialize('insetos.ttl')

<Graph identifier=Nce6f77d3a5df452c8bf7a7096de8d58c (<class 'rdflib.graph.Graph'>)>